In [1]:
import sqlite3
import polars as pl

pl.Config.set_tbl_cols(15)
pl.Config.set_tbl_rows(-1)
# 连接到 SQLite3 数据库
conn = sqlite3.connect("db/data.db")

# 执行查询
query = "SELECT * FROM trains"

# 使用 Polars 从 SQLite3 数据库中读取数据
df = pl.read_database(query, conn)
df = df.with_columns(
    (pl.col("time_agg_update") + pl.col("time_enc") + pl.col("time_dec")).alias(
        "time_crypt"
    ),
    (
        pl.col("time_local_train") + pl.col("time_calc_mask") + pl.col("time_agg_mask")
    ).alias("time_common"),
)

df = df.with_columns((pl.col("time_crypt") + pl.col("time_common")).alias("total_time"))


# 选择主键列、accuracy 和 total_time
result = df.select(
    [
        "algorithm",
        "strategy",
        "split",
        "ordinal",
        "round",
        "accuracy",
        "total_time",
    ]
)

result = result.sort(
    [
        "algorithm",
        "strategy",
        "split",
        "ordinal",
        "round",
    ]
)

# 按主键分组，计算accuracy的最大值和total_time的总值
result = result.groupby(
    ["algorithm", "strategy", "split", "ordinal"]
).agg(
    [
        pl.col("accuracy").max().alias("max_accuracy"),
        pl.col("total_time").sum().alias("total_time_sum"),
    ]
)

# 按主键排序
result = result.sort(
    [
        "algorithm",
        "strategy",
        "split",
        "ordinal",
    ]
)


# 打印 DataFrame
print(result)

# 关闭数据库连接
conn.close()


shape: (3_601, 7)
┌───────────────────┬──────────┬────────┬─────────┬───────┬──────────┬────────────┐
│ algorithm         ┆ strategy ┆ split  ┆ ordinal ┆ round ┆ accuracy ┆ total_time │
│ ---               ┆ ---      ┆ ---    ┆ ---     ┆ ---   ┆ ---      ┆ ---        │
│ str               ┆ str      ┆ str    ┆ i64     ┆ i64   ┆ f64      ┆ f64        │
╞═══════════════════╪══════════╪════════╪═════════╪═══════╪══════════╪════════════╡
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 0     ┆ 0.100382 ┆ 211.45786  │
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 1     ┆ 0.163312 ┆ 210.079165 │
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 2     ┆ 0.185478 ┆ 208.947841 │
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 3     ┆ 0.356943 ┆ 234.597365 │
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 4     ┆ 0.438726 ┆ 232.139746 │
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 5     ┆ 0.438726 ┆ 235.840343 │
│ ckksfedavg        ┆ max      ┆ dir0.5 ┆ 0       ┆ 6     